In [ ]:
from torchvision import transforms, datasets, models
import torch
from torch import optim
from torch.utils.data import DataLoader, sampler
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

import os
import numpy as np
import pandas as pd
import seaborn as sns

from PIL import Image #manipulations on image
from torchsummary import summary #
from timeit import default_timer as timer #timing

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 12

#to make the nb interactive
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
datadir = 'caltech'  #modify and unzip the data

In [ ]:
model = models.vgg16(pretrained =True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
for param in model.parameters():
  param.requires_grad = False

In [ ]:
num_inputs = model.classifier[6].in_features

#added customisation
model.classifier[6] = nn.Sequential(
    nn.Linear(num_inputs, 256),
    nn.ReLU(),
    nn.Dropout(p = 0.4),

    nn.Linear(256, n_classes),
    nn.LogSoftmax(dim = 1)
)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters()
                    if p.requires_grad)
print(f'{total_trainable_params:,} total trainable parameters.')

In [ ]:
if torch.cuda.is_available():
  model = model.cuda()

In [ ]:
def get_pretrained_model(model_name):

  if model_name == 'vgg16':
    model = models.vgg16(pretrained=True)

      # Freeze early layers
      for param in model.parameters():
          param.requires_grad = False
      n_inputs = model.classifier[6].in_features

      # Add on classifier
      model.classifier[6] = nn.Sequential(
          nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2),
          nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))

  elif model_name == 'resnet50':
    model = models.resnet50(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    n_inputs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2),
        nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))
    
    #in resnet50 we replace the fc layer with the custom classifier
   

In [ ]:
model = get_pretrained_model('vgg16')

summary(model,
        input_size = (3, 224, 224),
        batch_size = batch_size,
        device = 'cuda')

In [ ]:
print(model.classifier[6])

In [ ]:
model.class_to_idx = data['train'].class_to_idx
model.idx_to_class = {
    idx : class_
    for class_, idx in model.class_to_idx.items()   
}

list(model.class_to_idx.items())[:10]

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.NLLLoss()